**多层感知机十分适合处理表格数据，其中行对应样本，列对应特征**。然而对于高维感知数据，这种缺少结构的网络可能会变得不实用。

例如，在之前猫狗分类的例子中：假设我们有一个足够充分的照片数据集，数据集中是拥有标注的照片，每张照片具有百万级像素，这意味着网络的每次输入都有一百万个维度。即使将隐藏层维度降低到1000，这个全连接层也将有$10^6 \times 10^3 = 10^9$个参数。**想要训练这个模型将不可实现，因为需要有大量的GPU、分布式优化训练的经验和超乎常人的耐心**。

### 分类猫和狗的图片

- 使用一个还不错的相机采集图片（12M像素）
- RGB 图片有 36M 个像素点（元素）
- 使用 100 大小的**单隐藏层MLP**，需要 3.6B 个参数（14 GB）
  - 这远多于世界上所有猫和狗总数（900M 狗，600M 猫）

> [References](https://www.bilibili.com/video/BV1L64y1m7Nh/?share_source=copy_web&vd_source=a7ae9163cb2cd121bfd86ea1f4ecd2ef&t=112)


### 不变性

想象一下，**假设我们想从一张图片中找到某个物体。合理的假设是：无论哪种方法找到这个物体，都应该和物体的位置无关**。理想情况下，我们的系统应该能够利用常识：猪通常不在天上飞，飞机通常不在水里游泳。但是，如果一只猪出现在图片顶部，我们还是应该认出它。我们可以从儿童游戏“沃尔多在哪里”中得到灵感：在这个游戏中包含了许多充斥着活动的混乱场景，而沃尔多通常潜伏在一些不太可能的位置，读者的目标就是找出他。尽管沃尔多的装扮很有特点，但是在眼花缭乱的场景中找到他也如大海捞针。然而沃尔多的样子并不取决于他潜藏的地方，因此我们可以使用一个“沃尔多检测器”扫描图像。该检测器将图像分割成多个区域，并为每个区域包含沃尔多的可能性打分。卷积神经网络正是将**空间不变性**（spatial invariance）的这一概念系统化，从而基于这个模型使用较少的参数来学习有用的表示。

![沃尔多游戏示例图。](img/where-wally-walker-books.jpg)


总结一下，从设计适合于计算机视觉的神经网络架构：

1. **平移不变性**（translation invariance）：不管检测对象出现在图像中的哪个位置，神经网络的前面几层应该对相同的图像区域具有相似的反应，即为“平移不变性”。
2. **局部性**（locality）：神经网络的前面几层应该只探索输入图像中的局部区域，而不过度在意图像中相隔较远区域的关系，这就是“局部性”原则。最终，可以聚合这些局部特征，以在整个图像级别进行预测。

------------------------------

### **重新考察全连接层**

- 将输入和输出变形为矩阵（宽度，高度）
- **将权重变形为 4-D 张量 (h, w) 到 (h', w')**

$$h_{i,j} = \sum_{k,l}w_{i,j,k,l}x_{k,l} = \sum_{a,b}v_{i,j,a,b}x_{i+a,j+b}$$

- V 是 W 的重新索引$v_{i,j,a,b} = w_{i,j,i+a,j+b}$，**代入上式**

### **原则 #1 - 平移不变性**

- **x 的平移导致 h 的平移 $h_{i,j} = \sum_{a, b}v_{i,j,a,b}x_{i+a, j+b}$**
- v 不应该依赖于 (i, j)
- 解决方案：$v_{i,j,a,b} = v_{a,b}$
$$h_{i,j} = \sum_{a,b}x_{i+a,j+b}$$
> 这就是 2 维~~卷积~~ **交叉相关**

### **#2 - 局部性**

**$$h_{i,j} = \sum_{a,b}v_{a,b}x_{i+a,j+b}$$**

- 当评估$h_{i,j}$时，我们不应该用远离$x_{i,j}$的参数
- 解决方案：当$\left| a \right|,\left| b \right| > \Delta$时，使得$v_{a,b}=0$

$$h_{i,j} = \sum_{a=-\Delta}^{\Delta} \sum_{b=-\Delta}^{\Delta} v_{a,b}x_{i+a,j+b}$$

### 总结

- 对全连接层使用平移不变性和局部性得到卷积层

$$h_{i,j} = \sum_{a,b} v_{i,j,a,b} x_{i+a,j+b}$$
$$\Downarrow$$
$$h_{i,j} = \sum_{a = - \Delta}^{\Delta} \sum_{b = - \Delta}^{\Delta} v_{a,b} x_{i+a,j+b}$$